<a href="https://colab.research.google.com/github/LorenzoTarricone/Advnced-Programming-and-Optimization-Algorithm/blob/main/Tarricone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Programming assignment 2
Consider a set of pictures from 360° camera mounted inside a merry-go-round. They were taken at night and only one seat is visible which emits light – the seat in the shape of a jelly fish. We know that the merry-go-round rotates clockwise and that all the pictures were taken during a single cycle of merry-go-round. Given that the first picture is P1, your task is to sort the rest in the chronological order. Assume that the move of jelly fish is monotonous in horizontal direction, i.e., it always moves in clockwise direction, never backwards.

Input files: text files, each of them with 10 rows and 80 columns representing the brightness level in the given parts of the picture. Jelly fish can be recognized by high brightness (value 1 to 9) on a black background (value 0).

```
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000005950000000000000000000000000000000000000000
00000000000000000000000000000000000009990000000000000000000000000000000000000000
00000000000000000000000000000000000005950000000000000000000000000000000000000000
00000000000000000000000000000000000003930000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
```
###Directions
In order to capture the movement of the jelly fish between two pictures, use Earth-mover distance, also known as Wasserstein distance. It can be formulated as a min-cost flow problem on a certain network. Use NetworkX package (https://networkx.org) to perform the optimization required to find this flow.

About Earth-mover distance: Consider two distributions over a discrete set of points $P$: $μ$ in $[0,1]^P$ and $λ$ in $[0,1]^P$. Since both are distributions and sum of $μ(p)$ over all points in $P$ is $1$ (and the same holds for $λ$), there is surely a way to transform $μ$ into $λ$: we denote $f_{p,q}$ the probability mass transfered from point $p$ to $q$ for each pair of points $p,q$ from $P$, in order to transform $μ$ into $λ$:

For each $p$ in $P$: $μ(p) = \sum q f_{p,q}$ and for each $q$ in $P$: $λ(q) = Σ
\sum p f_{p,q}$.

Note that $f_{p,p}$ denotes the probability mass which remains at the same place.
Let $d(p,q)$ denote the distance between $p$ and $q$. Then, the cost of the transformation $f$ is

$\sum\limits_{p,q} d(p,q) f_{p,q}$

The earth-mover distance of μ and λ is then the cost of the cheapest transformation between $μ$ and $λ$.

---



##Solution

In [ ]:
import numpy as np
import networkx as nx

#We create a dict called matrix to store iteratively the 12 matrices 
matrix = {}

for index in range(1, 13):
    
    #Initialize each matrix as an np.array
    matrix[str(index)] = np.array([])
    
    with open(f"P{index}.txt") as file:
        for line in file:
            #We replace the end of line with nothing to avoid problems while parsing
            line = line.replace("\n", "")
            for char in line:
                #We iteratively append the numbers adding them to the columns 
                matrix[str(index)] = np.append(matrix[str(index)], int(char))
    
    #We reshape the matrix
    matrix[str(index)] = np.reshape(matrix[str(index)], (10, 80))
    #And we normalize using them lcm 
    if index == 10:
        matrix[str(index)] = matrix[str(index)] * 80
    else:
        matrix[str(index)] = matrix[str(index)] * 39   

#We will calculate the distances of the image from the one called P1

#We start by creating tuples of three elements where for the first matrix we save
#the value of the pixels different from zero and their position in the matrix 
l1 = []
auxiliary_lists = {}
auxiliary_graphs = {}
distances = {}

for i in range (10):
    for j in range (80):
        if matrix["1"][i][j] != 0:
            l1.append((matrix["1"][i][j],i,j))

#We do the same for all the other matrixes and then we use the column distance 
#as weight to calculate the min_cost_flow

for k in range (2, 13):
    
    auxiliary_lists[str(k)] = []
    
    for i in range (10):
        for j in range (80):
            if matrix[str(k)][i][j] != 0:
                auxiliary_lists[str(k)].append((matrix[str(k)][i][j],i,j)) 
                
    auxiliary_graphs[str(k)] = nx.DiGraph()
    
    for i in range (12):
        auxiliary_graphs[str(k)].add_node(l1[i], demand = -l1[i][0])
        auxiliary_graphs[str(k)].add_node(auxiliary_lists[str(k)][i], 
                                          demand =  auxiliary_lists[str(k)][i][0])
    
    for i in range (12):
        for j in range(12):
            auxiliary_graphs[str(k)].add_edge(l1[i], auxiliary_lists[str(k)][j], 
                                              weight = (auxiliary_lists[str(k)][j][2]-l1[i][2])%80)
    
    distances[str(k)] = nx.min_cost_flow_cost(auxiliary_graphs[str(k)])

#Nice visual representation of one of the graphs :)
nx.draw_shell(auxiliary_graphs["2"], node_color = "red")

#We eventually order and print the results     
print(1, end = " ")    
[print(key, end = " ") for (key, value) in sorted(distances.items() ,  key=lambda x: x[1]  ) ]
